# Lab 1.5

### 1. Troubleshooting Index Query #1



In [ ]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

ALTER PROC [dbo].[usp_Q7521] @UserId INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/7521/how-unsung-am-i */

-- How Unsung am I?
-- Zero and non-zero accepted count. Self-accepted answers do not count.

select
    count(a.Id) as [Accepted Answers],
    sum(case when a.Score = 0 then 0 else 1 end) as [Scored Answers],  
    sum(case when a.Score = 0 then 1 else 0 end) as [Unscored Answers],
    sum(CASE WHEN a.Score = 0 then 1 else 0 end)*1000 / count(a.Id) / 10.0 as [Percentage Unscored]
from
    Posts q
  inner join
    Posts a
  on a.Id = q.AcceptedAnswerId
where
      a.CommunityOwnedDate is null
  and a.OwnerUserId = @UserId
  and q.OwnerUserId != @UserId
  and a.PostTypeId = 2
END
GO


Brent introduces a strategy to identify the selectivity of a JOIN query by doing the following:

1. Extract both tables into seperate SELECT statements, including the where clause against the relevant tables
2. Add the JOIN condition to the WHERE clause, WHERE table.Id IN (Id's from Join Table)

Unpacking the above query, we have:

In [ ]:
SELECT * 
FROM dbo.Posts q
WHERE q.OwnerUserID = @UserId
AND a.PostTypeId = 2

SELECT *
FROM dbo.Posts a
WHERE 
    a.CommunityOwnedDate IS NULL
    AND a.OwnerUserID = @UserId
    AND a.PostTypeId = 2
    AND a.Id IN (...)

Next question: how selective are the fields in the WHERE clause?

We want to find which process is going to be executed first (ie. the one with the least overhead), and how much work that creates downstream.

In [ ]:
-- Find COUNT for the hardcoded values
SELECT COUNT(*) FROM dbo.Posts WHERE CommunityOwnedDate IS NULL;
SELECT COUNT() FROM dbo.Posts WHERE PostTypeId = 2;

-- Who are the users who own the most posts?
SELECT TOP 100 OwnerUserID, COUNT(*) AS recs
FROM dbo.Posts 
GROUP BY OwnerUserID
ORDER BY COUNT(*) DESC;

